# Goals

* Check on the test and/or production databases

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import os
import pandas as pd
from pypika import Query, Table, Field, Column, Criterion

In [3]:
from SRAgent.db.connect import db_connect
from SRAgent.db.upsert import db_upsert
from SRAgent.db.utils import db_list_tables, db_glimpse_tables, db_get_table, execute_query
from SRAgent.db.get import db_find_srx
from SRAgent.db.create import create_table, create_table_router

In [4]:
# list all of the tables in prod
os.environ['DYNACONF'] = 'prod'
conn_prod = db_connect() 
print("\n".join(db_list_tables(conn_prod)))

eval
screcounter_log
srx_srr
srx_metadata
screcounter_star_params
screcounter_star_results


# Production

In [7]:
db_list_tables(conn_prod)

['eval',
 'screcounter_log',
 'srx_srr',
 'srx_metadata',
 'screcounter_star_params',
 'screcounter_star_results']

In [5]:
db_glimpse_tables(conn_prod)

#-- Table: eval --#
dataset_id	database	entrez_id	srx_accession	is_illumina	is_single_cell	is_paired_end	lib_prep	tech_10x	organism	cell_prep	created_at	updated_at
eval1	sra	29115023	ERX11157721	yes	yes	yes	10x_Genomics	3_prime_gex	zebrafish	single_cell	2025-01-03 18:38:01.789148	2025-01-03 18:38:01.789148
eval1	sra	36178506	ERX11887200	yes	yes	yes	10x_Genomics	3_prime_gex	mouse	single_cell	2025-01-03 18:38:01.789148	2025-01-03 18:38:01.789148
eval1	sra	5920965	ERX2188110	yes	yes	yes	other	not_applicable	macaque	single_cell	2025-01-03 18:38:01.789148	2025-01-03 18:38:01.789148
eval1	sra	34439895	ERX9692805	yes	yes	yes	10x_Genomics	3_prime_gex	chicken	single_cell	2025-01-03 18:38:01.789148	2025-01-03 18:38:01.789148
eval1	gds	200271457	NaN	no	no	no	not_applicable	not_applicable	human	not_applicable	2025-01-03 18:38:01.789148	2025-01-03 18:38:01.789148

#-- Table: screcounter_log --#
sample	accession	process	step	status	message	created_at	updated_at

#-- Table: srx_srr --#
srx_accession	

In [6]:
# read in srx-metadata as pandas dataframe
tbl = Table("srx_metadata")
stmt = Query \
    .from_(tbl) \
    .select("*") \
    .distinct() \
    .where(tbl.notes != "Processed by Chris Carpenter")
srx_metadata = pd.read_sql(str(stmt), conn_prod)
srx_metadata

,database,entrez_id,srx_accession,is_illumina,is_single_cell,is_paired_end,lib_prep,tech_10x,cell_prep,organism,tissue,disease,purturbation,cell_line,czi_collection_id,czi_collection_name,notes,created_at,updated_at
0,sra,26330964,SRX19136133,yes,yes,yes,10x_Genomics,3_prime_gex,single_cell,mouse,Apical Ectodermal Ridge (AER),Split-Hand/Foot Malformation type 3 (SHFM3),Investigated functional consequences of SHFM3-...,other,None,None,Metadata obtained by SRAgent,2025-01-03 18:42:38.083215,2025-01-03 18:45:00.368512
1,sra,27694563,SRX20273630,yes,yes,yes,other,not_applicable,single_cell,human,HCT116,not specified,infected,HCT116,None,None,Metadata obtained by SRAgent,2025-01-03 18:42:38.083215,2025-01-03 18:44:17.048627
2,sra,29777569,SRX21883929,yes,yes,yes,10x_Genomics,3_prime_gex,single_cell,mouse,lymph node,"ovarian tumors, immune checkpoint blockade sen...",2-Deoxy-D-glucose (2DG),unsure,None,None,Metadata obtained by SRAgent,2025-01-03 18:42:38.083215,2025-01-03 18:44:08.060710
3,sra,29919592,SRX22006379,yes,yes,yes,Smart-seq2,not_applicable,single_cell,human,blastocyst,unsure,unsure,inner cell mass (ICM),None,None,Metadata obtained by SRAgent,2025-01-03 18:42:38.083215,2025-01-03 18:43:57.602382
4,sra,30865352,SRX22823407,yes,yes,yes,other,not_applicable,single_cell,mouse,tumor,phaeohyphomycosis,anti-PD1 treatment,endogenous CD8+ T cells,None,None,Metadata obtained by SRAgent,2025-01-03 18:42:38.083215,2025-01-03 18:45:48.003195
5,sra,33249222,SRX24914484,yes,yes,yes,Smart-seq2,not_applicable,single_cell,human,blood,hepatitis C virus (HCV),E2-specific B cells sorted from PBMCs,memory B cells,None,None,Metadata obtained by SRAgent,2025-01-03 18:42:38.083215,2025-01-03 18:43:48.934750
6,sra,34602788,SRX25602002,yes,yes,yes,other,not_applicable,single_cell,human,PBMCs,Type 2 Diabetes Mellitus,Randomized controlled study on salt intake eff...,other,None,None,Metadata obtained by SRAgent,2025-01-03 18:42:38.083215,2025-01-03 18:45:27.430411
7,sra,34648698,SRX25634198,yes,yes,yes,other,not_applicable,single_cell,mouse,prostate,prostate cancer,anti-PD1 + Vehicle,Myc-CaP,None,None,Metadata obtained by SRAgent,2025-01-03 18:42:38.083215,2025-01-03 18:44:30.489428
8,sra,34996766,SRX25926423,yes,yes,yes,10x_Genomics,3_prime_gex,single_cell,mouse,small intestine,graft vs. host disease (GVHD),STAT1 deletion in murine intestinal epithelium,unsure,None,None,Metadata obtained by SRAgent,2025-01-03 18:42:38.083215,2025-01-03 18:43:38.319328
9,sra,36086856,SRX26710454,yes,yes,yes,Smart-seq2,other,single_cell,mouse,D-LN (Duodenal Lymph Nodes),none,HP (Helminth Infection),none,None,None,Metadata obtained by SRAgent,2025-01-03 18:42:38.083215,2025-01-03 18:45:29.642017
